# Getting Started

## Data Harmonization with `bdikit`

Data harmonization is the process of integrating and aligning data from different sources into a consistent format to ensure compatibility and interoperability across data analyses and systems. `bdikit` is a library the helps with key data harmonization steps:
- *Schema Mapping*: In this step, data from various sources are mapped to a unified schema or model. This involves identifying equivalent table columns and establishing relationships between disparate datasets.
- *Value Mapping (Data Standardization)*: This step involves converting data into a common format or structure, using consistent naming conventions, units, and coding systems to ensure uniformity.

In this example, we describe how `bdikit` can be used to harmonize datasets from two papers:
- Dou et al. (https://pubmed.ncbi.nlm.nih.gov/37567170/)
- Cao et al (https://www.cell.com/cell/fulltext/S0092-8674(21)00997-1).

#### Loading the data

First, import the `bdikit` library and other libraries.

In [1]:
import bdikit as bdi
import pandas as pd
from IPython.display import display, Markdown

Next, we load our source data using Pandas and select some columns we are interested in.

In [2]:
df_source = pd.read_csv("./datasets/dou.csv")
column_names = [
    "Country",
    "Gender",
    "FIGO_stage",
    "Path_Stage_Reg_Lymph_Nodes-pN",
    "tumor_Stage-Pathological",
    "Tumor_Focality",
]
df_source = df_source[column_names]
df_source.head(10)

,Country,Gender,FIGO_stage,Path_Stage_Reg_Lymph_Nodes-pN,tumor_Stage-Pathological,Tumor_Focality
0,United States,Female,IA,pN0,Stage I,Unifocal
1,United States,Female,IA,pNX,Stage IV,Unifocal
2,United States,Female,IA,pN0,Stage I,Unifocal
3,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,Female,IA,pNX,Stage I,Unifocal
5,United States,Female,IA,pNX,Stage I,Unifocal
6,United States,Female,IA,pNX,Stage I,Unifocal
7,Other_specify,Female,IA,pNX,Stage I,Unifocal
8,United States,Female,IIIA,pNX,Stage III,Unifocal
9,United States,Female,IA,pN0,Stage I,Unifocal


Our goal is to harmonize the data from our source table (`dou.csv`) with the data from our target table `cao.csv`

In [3]:
df_target = pd.read_csv("./datasets/cao.csv")
df_target.head(5)

,case_id,tumor_included_for_the_study,normal_included_for_the_study,histology_diagnosis,age,sex,race,participant_country,tumor_site,tumor_focality,...,Islet_fraction,Stromal_fraction,Non_neoplastic_duct,Fat_fraction,Inflammation_fraction,Muscle_fraction,follow_up_days,vital_status,is_this_patient_lost_to_follow_up,cause_of_death
0,C3L-00102,yes,yes,PDAC,42,Male,White,United States,head,Unifocal,...,2,60,1,4,7,0,249.0,Deceased,No,pancreatic carcinoma
1,C3L-00189,yes,yes,PDAC,68,Female,NaN,Canada,head,Unifocal,...,1;3;2,75;55;53,2;1;0,0;6;0,7;30;25,0;0;0,1035.0,Deceased,No,pancreatic carcinoma
2,C3L-00277,yes,yes,PDAC,69,Male,White,Other,tail,Unifocal,...,0,55,0,0,5,0,610.0,Deceased,Yes,pancreatic carcinoma
3,C3L-00401,yes,yes,PDAC,62,Female,NaN,Canada,body,Unifocal,...,2;2;3,35;30;32,3;10;10,0;0;0,10;13;5,0;0;0,1228.0,Living,No,na
4,C3L-00640,yes,yes,PDAC,59,Female,White,United States,body,Unifocal,...,0;0,55;44,0;0,0;0,10;15,0;1,594.0,Living,No,na


#### Finding column matches between two tables

`bdi-kit` offers a suite of functions to help with data harmonization tasks.

For instance, it can help automatically discover one-to-one mappings between the source and target dataset columns.

To do so using `bdi-kit`, we can use the `match_schema()` function to match columns of the two schemas as follows.

In [4]:
schema_mapping = bdi.match_schema(df_source, df_target, method="ct_learning")
schema_mapping

  0%|          | 0/6 [00:00<?, ?it/s]

Table features extracted from 6 columns


  0%|          | 0/39 [00:00<?, ?it/s]

Table features extracted from 39 columns


,source,target
0,Country,participant_country
1,Gender,sex
2,FIGO_stage,tumor_stage_pathological
3,Path_Stage_Reg_Lymph_Nodes-pN,pathologic_staging_regional_lymph_nodes_pn
4,tumor_Stage-Pathological,tumor_stage_pathological
5,Tumor_Focality,tumor_focality


#### Finding value matches between two columns

Once the matching columns are identified, we can standardize data to ensure that no duplicate values represent the same entity/meaning.

To do that, `bdikit` provides the function `match_values()` to find values that should potentially be merged. The library supports multiple methods to perform this task, including syntactic and semantic matching algorithms. In this example, we use the `tfidf` method, which finds values based on the similarity of character n-grams. Please, refer to the [bdikit documentation](https://bdi-kit.readthedocs.io/) to learn more about the methods available.

In [5]:
# Matches values from each pair of source-target columns
value_matches = bdi.match_values(df_source, df_target, schema_mapping, method="tfidf")

# Print value matches
for match in value_matches:
    display(
        Markdown(
            f"<br>**Source column:** {match.attrs['source']}<br>"
            f"**Target column:** {match.attrs['target']}<br>"
        )
    )
    display(match)

<br>**Source column:** Country<br>**Target column:** participant_country<br>

,source,target,similarity
0,United States,United States,1.000
1,Poland,Poland,1.000
2,Other_specify,Other,0.543
3,nan,Canada,0.339
4,Ukraine,None,NaN


<br>**Source column:** Gender<br>**Target column:** sex<br>

,source,target,similarity
0,Female,Female,1.0
1,nan,None,NaN


<br>**Source column:** FIGO_stage<br>**Target column:** tumor_stage_pathological<br>

...

,source,target,similarity
0,pN0,pN0,1.000
1,pNX,pNX,1.000
2,pN2 (FIGO IIIC2),pN2,0.379
3,pN1 (FIGO IIIC1),pN1,0.379
4,nan,None,NaN


<br>**Source column:** tumor_Stage-Pathological<br>**Target column:** tumor_stage_pathological<br>

,source,target,similarity
0,Stage IV,Stage IV,1.000
1,nan,NaN,1.000
2,Stage III,Stage III,1.000
3,Stage II,Stage III,0.897
4,Stage I,Stage IA,0.857


<br>**Source column:** Tumor_Focality<br>**Target column:** tumor_focality<br>

,source,target,similarity
0,Unifocal,Unifocal,1.0
1,nan,NaN,1.0
2,Multifocal,Multifocal,1.0


#### Generating a harmonized table

In [6]:
df_mapped = bdi.materialize_mapping(df_source, value_matches)
df_mapped

,participant_country,sex,tumor_stage_pathological,pathologic_staging_regional_lymph_nodes_pn,tumor_focality
0,United States,Female,Stage IA,pN0,Unifocal
1,United States,Female,Stage IV,pNX,Unifocal
2,United States,Female,Stage IA,pN0,Unifocal
3,NaN,NaN,NaN,NaN,NaN
4,United States,Female,Stage IA,pNX,Unifocal
...,...,...,...,...,...
99,None,Female,Stage IA,pNX,Unifocal
100,None,Female,Stage III,pN0,Unifocal
101,United States,Female,Stage III,pN0,Unifocal
102,None,Female,Stage IA,pN0,Unifocal


In [7]:
source_column_names = list(map(lambda m: m.attrs['source'], value_matches))
target_column_names = list(map(lambda m: m.attrs['target'], value_matches))
df_source[source_column_names]

,Country,Gender,FIGO_stage,Path_Stage_Reg_Lymph_Nodes-pN,tumor_Stage-Pathological,Tumor_Focality
0,United States,Female,IA,pN0,Stage I,Unifocal
1,United States,Female,IA,pNX,Stage IV,Unifocal
2,United States,Female,IA,pN0,Stage I,Unifocal
3,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,Female,IA,pNX,Stage I,Unifocal
...,...,...,...,...,...,...
99,Ukraine,Female,IA,pNX,Stage I,Unifocal
100,Ukraine,Female,II,pN0,Stage II,Unifocal
101,United States,Female,II,pN0,Stage II,Unifocal
102,Ukraine,Female,IA,pN0,Stage I,Unifocal


In [8]:
pd.concat([df_mapped[target_column_names], df_target[target_column_names]])

,participant_country,sex,tumor_stage_pathological,pathologic_staging_regional_lymph_nodes_pn,tumor_stage_pathological,tumor_focality
0,United States,Female,Stage IA,pN0,Stage IA,Unifocal
1,United States,Female,Stage IV,pNX,Stage IV,Unifocal
2,United States,Female,Stage IA,pN0,Stage IA,Unifocal
3,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,Female,Stage IA,pNX,Stage IA,Unifocal
...,...,...,...,...,...,...
135,Poland,Male,Stage III,pN2,Stage III,Unifocal
136,China,Female,Stage III,pN2,Stage III,Unifocal
137,China,Male,Stage III,pN2,Stage III,Unifocal
138,Poland,Female,Stage III,pN2,Stage III,Multifocal
